In [1]:
import collections
import datetime
import itertools
import math
import operator
import random
import time

import catboost
import hyperopt
import numpy
import pandas
from scipy import stats
from sklearn import (
    calibration,
    ensemble,
    linear_model,
    metrics,
    model_selection,
    naive_bayes,
    neighbors,
    neural_network,
    svm,
)
import xgboost

/Users/eigenein/Library/Python/3.6/lib/python/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Read and Filter

In [2]:
dtype = {'smoke': float, 'alco': float, 'active': float, 'ap_hi': float, 'ap_lo': float}

def fix_ap(value):
    value = abs(value)
    while value > 500.0:
        value /= 10.0
    return value

def read_csv(filename):
    frame = pandas.read_csv(filename, sep=';', header=0, na_values='None', dtype=dtype).drop(['id'], axis=1)
    
    frame = pandas.get_dummies(frame, columns=[
        # 'smoke',
        # 'active',
        # 'alco',
        # 'gender',
        # 'cholesterol',
        # 'gluc',
    ])
    frame = frame.assign(
        bmi=(frame['weight'] / frame['height'] ** 2),
        # aged_smoke_0=(frame['smoke_0.0'] * frame['age']),
        # aged_smoke_1=(frame['smoke_1.0'] * frame['age']),
        # aged_active_0=(frame['active_0.0'] * frame['age']),
        # aged_active_1=(frame['active_1.0'] * frame['age']),
        # aged_alco_0=(frame['alco_0.0'] * frame['age']),
        # aged_alco_1=(frame['alco_1.0'] * frame['age']),
        # smoke_1_active_0=(frame['smoke_1.0'] * frame['active_0.0']),
    )
    
    # Fix negative and too large values.
    frame['ap_hi'] = frame['ap_hi'].apply(fix_ap)
    frame['ap_lo'] = frame['ap_lo'].apply(fix_ap)
    
    # Re-order ap_hi and ap_lo.
    frame[['ap_hi', 'ap_lo']] = frame[['ap_hi', 'ap_lo']].apply(
        lambda row: [row['ap_hi'], row['ap_lo']] if row['ap_hi'] > row['ap_lo'] else [row['ap_lo'], row['ap_hi']],
        axis=1,
    )
    
    return frame

In [3]:
train = read_csv('train.csv')

X = train.drop('cardio', axis=1).values
y = train['cardio'].values
print(f'X: {X.shape}')
print(f'y: {y.shape}')

X: (70000, 12)
y: (70000,)


In [50]:
test = read_csv('test.csv')

test['smoke'].fillna(0.0, inplace=True)
test['alco'].fillna(0.0, inplace=True)
test['active'].fillna(1.0, inplace=True)

X_test = test.values

### Hyper-parameter Optimisation

In [4]:
def cv():
    return model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [18]:
Param = collections.namedtuple('Param', 'min max step')

def optimize_estimator(estimator, X, y, n_iter=10, scoring=None, refit=True, cv=None, fit_params=None, **params):
    # FIXME: rename `params` to `hyperparams` and make it a normal parameter.
    
    def evaluate():
        return model_selection.cross_val_score(estimator, X, y, scoring=scoring, cv=cv(), fit_params=fit_params).mean()
    
    def set_evaluate(key, value):
        setattr(estimator, key, value)
        return evaluate()
    
    params = list(params.items())
    
    print(f'Evaluating current model…')
    current_score = evaluate()
    print(f'Initial score: {current_score}')
    print()
    
    for i in range(n_iter):
        start_time = time.time()
        early_stop = True
        
        # Shuffle params to lessen overfitting.
        random.shuffle(params)
        
        # Tune each parameter.
        for key, param in params:
            current_value = getattr(estimator, key)
            
            # Choose the best value.
            values = [current_value - param.step, current_value + param.step]
            scores = [(value, set_evaluate(key, value)) for value in values if param.min <= value <= param.max]
            best_value, best_score = max(scores, key=operator.itemgetter(1))
            if best_score > current_score:
                current_value = best_value
                current_score = best_score
                early_stop = False
                print(f'[Iteration {i}] {key} = {current_value}. Score: {current_score}')
                
            # Restore current value.
            setattr(estimator, key, current_value)
        
        print(f'[Iteration {i}] Finished in {(time.time() - start_time) / 60.0:.0f} min')
        
        if early_stop:
            print(f'[Iteration {i}] No changes, stopping')
            break
            
    print(f'Final score: {current_score}')

    # Fit on the entire dataset if needed.
    if refit:
        print(f'Final fitting…')
        estimator.fit(X, y)
    return estimator

### Predict `cardio`

In [59]:
y_results = []
for _ in range(10):
    X_train, X_eval, y_train, y_eval = model_selection.train_test_split(X, y, test_size=0.1)
    y_test = catboost.CatBoostClassifier(
        iterations=1000, 
        thread_count=5, 
        random_seed=0,
        eval_metric='Logloss',
        use_best_model=True,
        verbose=True,
    ).fit(X_train, y_train, cat_features=[1, 6, 7, 8, 9, 10], eval_set=(X_eval, y_eval)).predict_proba(X_test)[:, 1]
    y_results.append(y_test)

Borders generated
0:	learn 0.684226773	test 0.6841080316	bestTest 0.6841080316	passed: 0.195 sec	total: 334ms	remaining: 5m 33s
1:	learn 0.6758877045	test 0.675701715	bestTest 0.675701715	passed: 0.179 sec	total: 513ms	remaining: 4m 16s
2:	learn 0.668300368	test 0.667953894	bestTest 0.667953894	passed: 0.16 sec	total: 673ms	remaining: 3m 43s
3:	learn 0.6611040364	test 0.6606764433	bestTest 0.6606764433	passed: 0.182 sec	total: 855ms	remaining: 3m 32s
4:	learn 0.6542702206	test 0.6537200612	bestTest 0.6537200612	passed: 0.18 sec	total: 1.03s	remaining: 3m 25s
5:	learn 0.648023771	test 0.6473958538	bestTest 0.6473958538	passed: 0.188 sec	total: 1.22s	remaining: 3m 22s
6:	learn 0.6421286692	test 0.6413571936	bestTest 0.6413571936	passed: 0.209 sec	total: 1.43s	remaining: 3m 23s
7:	learn 0.6366821239	test 0.6358167305	bestTest 0.6358167305	passed: 0.171 sec	total: 1.6s	remaining: 3m 18s
8:	learn 0.6312802717	test 0.6303957441	bestTest 0.6303957441	passed: 0.17 sec	total: 1.77s	remaining: 3

75:	learn 0.5453053259	test 0.5441769835	bestTest 0.5441769835	passed: 0.184 sec	total: 13.2s	remaining: 2m 40s
76:	learn 0.5451129669	test 0.5439703842	bestTest 0.5439703842	passed: 0.189 sec	total: 13.4s	remaining: 2m 40s
77:	learn 0.5449033807	test 0.54378617	bestTest 0.54378617	passed: 0.171 sec	total: 13.6s	remaining: 2m 40s
78:	learn 0.5447165933	test 0.5436376664	bestTest 0.5436376664	passed: 0.203 sec	total: 13.8s	remaining: 2m 40s
79:	learn 0.5445149497	test 0.5434225491	bestTest 0.5434225491	passed: 0.17 sec	total: 14s	remaining: 2m 40s
80:	learn 0.5442978445	test 0.5432355801	bestTest 0.5432355801	passed: 0.174 sec	total: 14.1s	remaining: 2m 40s
81:	learn 0.5441708122	test 0.5431001305	bestTest 0.5431001305	passed: 0.123 sec	total: 14.3s	remaining: 2m 39s
82:	learn 0.544015917	test 0.5429704267	bestTest 0.5429704267	passed: 0.157 sec	total: 14.4s	remaining: 2m 39s
83:	learn 0.5438362211	test 0.5428076043	bestTest 0.5428076043	passed: 0.177 sec	total: 14.6s	remaining: 2m 39s


149:	learn 0.5390931247	test 0.5394083098	bestTest 0.5394083098	passed: 0.163 sec	total: 26s	remaining: 2m 27s
150:	learn 0.5390724546	test 0.539397521	bestTest 0.539397521	passed: 0.128 sec	total: 26.1s	remaining: 2m 26s
151:	learn 0.5390183444	test 0.5393758897	bestTest 0.5393758897	passed: 0.186 sec	total: 26.3s	remaining: 2m 26s
152:	learn 0.5389909401	test 0.5393479111	bestTest 0.5393479111	passed: 0.227 sec	total: 26.5s	remaining: 2m 26s
153:	learn 0.5389536878	test 0.5393096456	bestTest 0.5393096456	passed: 0.222 sec	total: 26.8s	remaining: 2m 27s
154:	learn 0.5389107684	test 0.5392697491	bestTest 0.5392697491	passed: 0.192 sec	total: 27s	remaining: 2m 26s
155:	learn 0.5388815535	test 0.539249871	bestTest 0.539249871	passed: 0.199 sec	total: 27.2s	remaining: 2m 26s
156:	learn 0.5388256789	test 0.5392252924	bestTest 0.5392252924	passed: 0.207 sec	total: 27.4s	remaining: 2m 26s
157:	learn 0.5387995942	test 0.539206327	bestTest 0.539206327	passed: 0.185 sec	total: 27.6s	remaining: 

222:	learn 0.537283763	test 0.5386422151	bestTest 0.5386422151	passed: 0.211 sec	total: 39.5s	remaining: 2m 17s
223:	learn 0.5372720928	test 0.5386521748	bestTest 0.5386422151	passed: 0.204 sec	total: 39.7s	remaining: 2m 17s
224:	learn 0.5372470695	test 0.5386411971	bestTest 0.5386411971	passed: 0.218 sec	total: 40s	remaining: 2m 17s
225:	learn 0.5372398811	test 0.538639399	bestTest 0.538639399	passed: 0.198 sec	total: 40.2s	remaining: 2m 17s
226:	learn 0.5372280325	test 0.5386414027	bestTest 0.538639399	passed: 0.104 sec	total: 40.3s	remaining: 2m 17s
227:	learn 0.5372136885	test 0.5386367491	bestTest 0.5386367491	passed: 0.203 sec	total: 40.5s	remaining: 2m 16s
228:	learn 0.5371849468	test 0.5386280338	bestTest 0.5386280338	passed: 0.199 sec	total: 40.7s	remaining: 2m 16s
229:	learn 0.5371800787	test 0.5386362539	bestTest 0.5386280338	passed: 0.205 sec	total: 40.9s	remaining: 2m 16s
230:	learn 0.5371742929	test 0.5386369783	bestTest 0.5386280338	passed: 0.114 sec	total: 41s	remaining

296:	learn 0.5362848466	test 0.5382791915	bestTest 0.5382745594	passed: 0.201 sec	total: 53.1s	remaining: 2m 5s
297:	learn 0.5362705733	test 0.5382879362	bestTest 0.5382745594	passed: 0.211 sec	total: 53.3s	remaining: 2m 5s
298:	learn 0.5362601387	test 0.5382871541	bestTest 0.5382745594	passed: 0.196 sec	total: 53.5s	remaining: 2m 5s
299:	learn 0.5362536882	test 0.5382822193	bestTest 0.5382745594	passed: 0.2 sec	total: 53.7s	remaining: 2m 5s
300:	learn 0.5362486126	test 0.538280859	bestTest 0.5382745594	passed: 0.178 sec	total: 53.9s	remaining: 2m 5s
301:	learn 0.5362290095	test 0.5382860273	bestTest 0.5382745594	passed: 0.201 sec	total: 54.1s	remaining: 2m 4s
302:	learn 0.5362252654	test 0.5382810449	bestTest 0.5382745594	passed: 0.138 sec	total: 54.2s	remaining: 2m 4s
303:	learn 0.5362232353	test 0.5382813968	bestTest 0.5382745594	passed: 0.0916 sec	total: 54.3s	remaining: 2m 4s
304:	learn 0.5361939815	test 0.538283132	bestTest 0.5382745594	passed: 0.181 sec	total: 54.5s	remaining: 2

...

655:	learn 0.5301227842	test 0.5394882617	bestTest 0.5394763125	passed: 0.212 sec	total: 1m 59s	remaining: 1m 2s
656:	learn 0.5301119579	test 0.5394789617	bestTest 0.5394763125	passed: 0.161 sec	total: 1m 59s	remaining: 1m 2s
657:	learn 0.530102763	test 0.5394749073	bestTest 0.5394749073	passed: 0.197 sec	total: 1m 59s	remaining: 1m 2s
658:	learn 0.5300901349	test 0.5394829606	bestTest 0.5394749073	passed: 0.181 sec	total: 2m	remaining: 1m 2s
659:	learn 0.5300796064	test 0.5394785332	bestTest 0.5394749073	passed: 0.185 sec	total: 2m	remaining: 1m 1s
660:	learn 0.5300635987	test 0.5394699366	bestTest 0.5394699366	passed: 0.206 sec	total: 2m	remaining: 1m 1s
661:	learn 0.5300487434	test 0.5394622081	bestTest 0.5394622081	passed: 0.19 sec	total: 2m	remaining: 1m 1s
662:	learn 0.5300408338	test 0.5394610738	bestTest 0.5394610738	passed: 0.183 sec	total: 2m	remaining: 1m 1s
663:	learn 0.5300281432	test 0.5394653374	bestTest 0.5394610738	passed: 0.197 sec	total: 2m 1s	remaining: 1m 1s
664:	l

729:	learn 0.5290921422	test 0.5393770467	bestTest 0.5393034734	passed: 0.211 sec	total: 2m 13s	remaining: 49.5s
730:	learn 0.529083209	test 0.5393676129	bestTest 0.5393034734	passed: 0.191 sec	total: 2m 13s	remaining: 49.3s
731:	learn 0.5290622157	test 0.5393756257	bestTest 0.5393034734	passed: 0.196 sec	total: 2m 14s	remaining: 49.1s
732:	learn 0.5290519045	test 0.5393822647	bestTest 0.5393034734	passed: 0.169 sec	total: 2m 14s	remaining: 48.9s
733:	learn 0.5290302106	test 0.5393675878	bestTest 0.5393034734	passed: 0.177 sec	total: 2m 14s	remaining: 48.8s
734:	learn 0.5290156118	test 0.5393730034	bestTest 0.5393034734	passed: 0.168 sec	total: 2m 14s	remaining: 48.6s
735:	learn 0.5289989155	test 0.5393670421	bestTest 0.5393034734	passed: 0.154 sec	total: 2m 14s	remaining: 48.4s
736:	learn 0.5289888772	test 0.5393630419	bestTest 0.5393034734	passed: 0.209 sec	total: 2m 15s	remaining: 48.2s
737:	learn 0.5289799615	test 0.5393633526	bestTest 0.5393034734	passed: 0.22 sec	total: 2m 15s	re

802:	learn 0.5280434925	test 0.5392428594	bestTest 0.5392329935	passed: 0.235 sec	total: 2m 27s	remaining: 36.3s
803:	learn 0.5280287522	test 0.5392459272	bestTest 0.5392329935	passed: 0.199 sec	total: 2m 28s	remaining: 36.1s
804:	learn 0.528010459	test 0.5392350579	bestTest 0.5392329935	passed: 0.202 sec	total: 2m 28s	remaining: 35.9s
805:	learn 0.5279999888	test 0.5392314203	bestTest 0.5392314203	passed: 0.176 sec	total: 2m 28s	remaining: 35.7s
806:	learn 0.5279876745	test 0.5392341924	bestTest 0.5392314203	passed: 0.189 sec	total: 2m 28s	remaining: 35.5s
807:	learn 0.5279650519	test 0.5392333716	bestTest 0.5392314203	passed: 0.206 sec	total: 2m 28s	remaining: 35.4s
808:	learn 0.5279448644	test 0.5392247898	bestTest 0.5392247898	passed: 0.184 sec	total: 2m 28s	remaining: 35.2s
809:	learn 0.5279311386	test 0.5392252544	bestTest 0.5392247898	passed: 0.179 sec	total: 2m 29s	remaining: 35s
810:	learn 0.5279151681	test 0.5392235741	bestTest 0.5392235741	passed: 0.16 sec	total: 2m 29s	rema

875:	learn 0.5270355947	test 0.5390953452	bestTest 0.5390684647	passed: 0.177 sec	total: 2m 42s	remaining: 23s
876:	learn 0.5270268425	test 0.5391011842	bestTest 0.5390684647	passed: 0.215 sec	total: 2m 42s	remaining: 22.8s
877:	learn 0.5270141132	test 0.5391007219	bestTest 0.5390684647	passed: 0.218 sec	total: 2m 42s	remaining: 22.6s
878:	learn 0.5269824749	test 0.539107504	bestTest 0.5390684647	passed: 0.158 sec	total: 2m 42s	remaining: 22.4s
879:	learn 0.5269746666	test 0.5390993968	bestTest 0.5390684647	passed: 0.185 sec	total: 2m 42s	remaining: 22.2s
880:	learn 0.5269560983	test 0.5390888065	bestTest 0.5390684647	passed: 0.177 sec	total: 2m 43s	remaining: 22s
881:	learn 0.5269512341	test 0.5390900425	bestTest 0.5390684647	passed: 0.167 sec	total: 2m 43s	remaining: 21.9s
882:	learn 0.5269414966	test 0.5390907364	bestTest 0.5390684647	passed: 0.184 sec	total: 2m 43s	remaining: 21.7s
883:	learn 0.5269275052	test 0.5390977729	bestTest 0.5390684647	passed: 0.186 sec	total: 2m 43s	remai

949:	learn 0.5260978101	test 0.539123269	bestTest 0.5390482938	passed: 0.189 sec	total: 2m 56s	remaining: 9.28s
950:	learn 0.5260864456	test 0.539112279	bestTest 0.5390482938	passed: 0.189 sec	total: 2m 56s	remaining: 9.09s
951:	learn 0.5260683537	test 0.5391027441	bestTest 0.5390482938	passed: 0.193 sec	total: 2m 56s	remaining: 8.91s
952:	learn 0.5260540073	test 0.5390974572	bestTest 0.5390482938	passed: 0.166 sec	total: 2m 56s	remaining: 8.72s
953:	learn 0.5260494002	test 0.5390975002	bestTest 0.5390482938	passed: 0.186 sec	total: 2m 57s	remaining: 8.54s
954:	learn 0.5260194421	test 0.5391008723	bestTest 0.5390482938	passed: 0.184 sec	total: 2m 57s	remaining: 8.35s
955:	learn 0.5260095874	test 0.5391076242	bestTest 0.5390482938	passed: 0.175 sec	total: 2m 57s	remaining: 8.16s
956:	learn 0.5260026481	test 0.5391003118	bestTest 0.5390482938	passed: 0.195 sec	total: 2m 57s	remaining: 7.98s
957:	learn 0.525991395	test 0.5391050558	bestTest 0.5390482938	passed: 0.177 sec	total: 2m 57s	rem

In [60]:
numpy.savetxt(f'results.txt', numpy.vstack(y_results).mean(axis=0), fmt='%f')